In [1]:
#in the below code we mainly geneated multi-view images,pointclouds,mesh and voxel with one-other modalities among which 
#                      image and poincloud modaliti used for retirval task on OS-mN40-MISS DATASET

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2022-02-25 13:06:29,797 - utils - NumExpr defaulting to 8 threads.


In [ ]:
#for query and target dataset create-folder as query1 and target1 and move query and target dataset respective folders for train 
#there is no need create a seperate folder
#for train
path="./OS-MN40-Miss/train/"
#for query data
path="./OS-MN40-MIss/query1/"
#for target data
path="./OS-MN40-MIss/target1/"

#set path based on the name of folder in dataset(train,query,target)

In [ ]:
import os
import numpy as np
import open3d as o3d
import random
import pymeshlab 
from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import binvox_rw
from pathlib import Path

In [3]:
def save_pic(mesh_r, i,path):   
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=512, height=512, visible = False)
    vis.get_render_option().point_color_option = o3d.visualization.PointColorOption.Color
    vis.get_render_option().point_size = 3.0
    vis.get_render_option().background_color = np.asarray([0,0,0])
    vis.add_geometry(mesh_r,reset_bounding_box=True)
    vis.capture_screen_image(f"{path}/h_{i}" +".png", do_render=True)
    vis.destroy_window()


In [ ]:
#converting mesh to pointcloud
class PointSampler(object):
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size
    
    def triangle_area(self, pt1, pt2, pt3):
        side_a = np.linalg.norm(pt1 - pt2)
        side_b = np.linalg.norm(pt2 - pt3)
        side_c = np.linalg.norm(pt3 - pt1)
        s = 0.5 * ( side_a + side_b + side_c)
        return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

    def sample_point(self, pt1, pt2, pt3):
        # barycentric coordinates on a triangle
        # https://mathworld.wolfram.com/BarycentricCoordinates.html
        s, t = sorted([random.random(), random.random()])
        f = lambda i: s * pt1[i] + (t-s)*pt2[i] + (1-t)*pt3[i]
        return (f(0), f(1), f(2))
        
    
    def __call__(self, mesh):
        verts, faces = mesh
        verts = np.array(verts)
        areas = np.zeros((len(faces)))

        for i in range(len(areas)):
            areas[i] = (self.triangle_area(verts[faces[i][0]],
                                           verts[faces[i][1]],
                                           verts[faces[i][2]]))
            
        sampled_faces = (random.choices(faces, 
                                      weights=areas,
                                      cum_weights=None,
                                      k=self.output_size))
        
        sampled_points = np.zeros((self.output_size, 3))

        for i in range(len(sampled_faces)):
            sampled_points[i] = (self.sample_point(verts[sampled_faces[i][0]],
                                                   verts[sampled_faces[i][1]],
                                                   verts[sampled_faces[i][2]]))
        
        return sampled_points
mesh = o3d.io.read_triangle_mesh("./sink/mesh/mesh.obj")
pointcloud = PointSampler(1048)((mesh.vertices, mesh.triangles))
a_file = open("sink1.txt", "w")
for row in pointcloud:
    for ele in row:
        a_file.write(str(ele))
        a_file.write(" ")
    a_file.write("\n")
a_file.close()



In [34]:
#all dataset list log file creation
#configure path for train,query,target

path="./OS-MN40-Miss/target1/"
os.mkdir("log_file")
a_file=open("./log_file/all_dataset_list/all_target_list","w")
a_file.write("Filename\t\t\t\t\t\t\t\t\timage\t\tmesh\t\tpointcloud\t\tvoxel\n")
cnt1=cnt2=cnt3=cnt4=0
for file in os.listdir(path):
     for file2 in os.listdir(path+file):
            a_file.write(path+file+'/'+ file2+"\t\t")
            if os.path.isdir(path+file+"/"+file2+"/image")==True:
                a_file.write("True\t\t")
            else:
                a_file.write("False\t\t")
                cnt1=cnt1+1
            if os.path.isdir(path+file+"/"+file2+"/mesh")==True:
                a_file.write("True\t\t")
            else:
                a_file.write("False\t\t")
                cnt2=cnt2+1
            if os.path.isdir(path+file+"/"+file2+"/pointcloud")==True:
                a_file.write("True\t\t")
            else:
                a_file.write("False\t\t")
                cnt3=cnt3+1
            if os.path.isdir(path+file+"/"+file2+"/voxel")==True:
                a_file.write("True\t\t")
            else:
                a_file.write("False\t\t")
                cnt4=cnt4+1
            a_file.write("\n")

            
#a_file.write(str(cnt1)+"  "+str(cnt2)+"  "+str(cnt3)+"  "+str(cnt4))
a_file.close()

#number of missing images,mesh,pointclud,voxels respectively in each dataset
print(cnt1,cnt2,cnt3,cnt4)
#1085 1053 1096 1044

374 355 361 368


In [65]:
#run all codes as in given order
#point cloud   to multi-view images with pointcloud generated mesh using solidwireframe fileter of pymeshlab
print("images generated are")
cnt=0
for file in os.listdir(path):
     for file2 in os.listdir(path+file):
            if os.path.isdir(path+file+"/"+file2+"/pointcloud")==True and os.path.isdir(path+file+"/"+file2+"/image")==False and os.path.isdir(path+file+"/"+file2+"/mesh")==False:
                cnt=cnt+1 
                #conversion code ---------#
                ms = pymeshlab.MeshSet()
                ms.load_new_mesh(path+file+"/"+file2+"/pointcloud/pt_2048.txt",strformat='X Y Z' ,separator='SPACE')
                
                #ms.load_filter_script("point_to_image.mlx")#enter script here script (create solid fram)
                
                ms.surface_reconstruction_ball_pivoting()
                ms.create_solid_wireframe()
                ms.set_current_mesh(new_curr_id=1)
                ms.save_current_mesh("createdmesh1.obj")
                os.mkdir(path+file+"/"+file2+"/image")
                print(path+file+"/"+file2+"/image")
        
                for i in range(24):
                    mesh2 = o3d.io.read_triangle_mesh('./createdmesh1.obj')
                    mesh3 = o3d.io.read_triangle_mesh('./createdmesh1.obj')
                    mesh2.compute_vertex_normals()
                    mesh3.compute_vertex_normals()
                    R1 = mesh2.get_rotation_matrix_from_xyz((0, 0, -0.5*(np.pi)))
                    mesh3.rotate(R1, center = (0, 0, 0))
                    #o3d.visualization.draw_geometries([mesh3])
                    R2 = mesh2.get_rotation_matrix_from_xyz(( -0.5*(np.pi), 0, 0))
                    mesh3.rotate(R2, center = (0, 0, 0))
                    #o3d.visualization.draw_geometries([mesh3])
                    R3 = mesh2.get_rotation_matrix_from_xyz((0, -2*i*(np.pi/24), 0))
                    mesh3.rotate(R3, center = (0, 0, 0))
                    #o3d.visualization.draw_geometries([mesh3])
                    R4 = mesh2.get_rotation_matrix_from_xyz((0.1666*(np.pi), 0, 0))
                    mesh3.rotate(R4, center = (0, 0, 0))
                    #o3d.visualization.draw_geometries([mesh3])
                    save_pic(mesh3,i,path+file+"/"+file2+"/image")
                   
        
print(f"total images generated are {cnt}")

../data/OS-MN40-Miss/target1/target/21b8d0032a1d15d3dd0a6b907aff49b2/image
../data/OS-MN40-Miss/target1/target/9c49f88bb21b2ca309032da0258bbd57/image
../data/OS-MN40-Miss/target1/target/849e537bdf921c3edfbd9b36a1e23876/image
../data/OS-MN40-Miss/target1/target/058283ae875b49dbd4764dfbc992f775/image
../data/OS-MN40-Miss/target1/target/c13d0d5682fa2f29b2de802ac44335cd/image
../data/OS-MN40-Miss/target1/target/7f88a5a3163f4954d5bbb69dbd02fc7f/image
../data/OS-MN40-Miss/target1/target/9a88c2fda78a9b3f0031cfe12db9a5fb/image
../data/OS-MN40-Miss/target1/target/5ccfdb6ca3e47118e99f5739553ef7ea/image
../data/OS-MN40-Miss/target1/target/158b238091557c09b7d600f45476fe4a/image
../data/OS-MN40-Miss/target1/target/2ed7fed328e2448ed8238e914017ac5a/image
../data/OS-MN40-Miss/target1/target/f8ab0724e5464d62f59169aae1d5a6c5/image
../data/OS-MN40-Miss/target1/target/9ef3ae498526f8840c253efdaa44c661/image
../data/OS-MN40-Miss/target1/target/349ac5ab0f96cffe732d0917a52aeefb/image
../data/OS-MN40-Miss/targ

../data/OS-MN40-Miss/target1/target/f78c4e8419bf8611fe17e681fce320e8/image
../data/OS-MN40-Miss/target1/target/0235a329378a46797f5ad938405c5330/image
../data/OS-MN40-Miss/target1/target/5803a30217a6d95d9548c6db2fc3ea3b/image
../data/OS-MN40-Miss/target1/target/31ca888473dced4bce36aada12695646/image
../data/OS-MN40-Miss/target1/target/3a8a46d3ef2cb12ca695d300ba6d47f3/image
../data/OS-MN40-Miss/target1/target/ede44cb85a312079e8210ff63ec6a30f/image
../data/OS-MN40-Miss/target1/target/89273a9e27149b336ce626a4af890085/image
../data/OS-MN40-Miss/target1/target/4becd038074e43f0aca5fc3e5ea9a548/image
../data/OS-MN40-Miss/target1/target/8dab974f126afa906f2e7d2cdfc3f339/image
../data/OS-MN40-Miss/target1/target/d41f6d622331cb025b2ceb87b18468a4/image
../data/OS-MN40-Miss/target1/target/f8b71682a816d8af53f79b26f41652f5/image
../data/OS-MN40-Miss/target1/target/08e8e633f91d50f7737015c81d0c06f5/image
../data/OS-MN40-Miss/target1/target/da1ab2d98263f10ed4b35bd9070801fe/image
../data/OS-MN40-Miss/targ

../data/OS-MN40-Miss/target1/target/8f126cbaeb8aef20bf59bcbd3d5e29bc/image
../data/OS-MN40-Miss/target1/target/1fb460f42dc1915288313fc322973752/image
../data/OS-MN40-Miss/target1/target/70aa00643d23b582bdf047d4d7c6df1f/image
../data/OS-MN40-Miss/target1/target/798186bc47d2f0c6bde8fc809491d243/image
../data/OS-MN40-Miss/target1/target/9611d5d575e98b6329e067146566aab8/image
../data/OS-MN40-Miss/target1/target/eaadc5685886fe94dfc8f1750dafd2ce/image
../data/OS-MN40-Miss/target1/target/2e147872b91919f60a20dd65f21b683c/image
../data/OS-MN40-Miss/target1/target/e97f45abaf2a90ba3905fe424eaa9d0c/image
../data/OS-MN40-Miss/target1/target/dc389f105cd6101c9484a4e97f3768ec/image
../data/OS-MN40-Miss/target1/target/97104bdd5971fe5a0a70175b88bbce8c/image
../data/OS-MN40-Miss/target1/target/c3aa96688df12b924fcd36b288d64f33/image
../data/OS-MN40-Miss/target1/target/8f2be78542df53bab477904486d825d0/image
../data/OS-MN40-Miss/target1/target/d0ec626e1953e274e83269a2fad6dfc5/image
../data/OS-MN40-Miss/targ

../data/OS-MN40-Miss/target1/target/46ec5cc1e2a235903de39c6a68767351/image
../data/OS-MN40-Miss/target1/target/f2e55cd2712606e9c3f97aebea7af278/image
../data/OS-MN40-Miss/target1/target/500809aff131a27929e45eb0a162ee0c/image
../data/OS-MN40-Miss/target1/target/7632b27fbae7f77e54c410eff87bcf4a/image
../data/OS-MN40-Miss/target1/target/e6c677b585e94595f03c32263038bb3b/image
../data/OS-MN40-Miss/target1/target/91a3ee2a985af3f5e60066b93d9d78d2/image
../data/OS-MN40-Miss/target1/target/69d8f1cd022f4d7cd9e88ccf8b797ce8/image
../data/OS-MN40-Miss/target1/target/55af308b34814441d9a905a5c4d9e6e3/image
../data/OS-MN40-Miss/target1/target/f1351ada93987a6c16339c6e1c518c6e/image
../data/OS-MN40-Miss/target1/target/8b4c5fe94689958e86c4975da69a2ce1/image
../data/OS-MN40-Miss/target1/target/cc43475074b3972b97ef7433d0ab500c/image
../data/OS-MN40-Miss/target1/target/93d1203eb5af6a1e614da4ba98931fe5/image
../data/OS-MN40-Miss/target1/target/15b0987f03afbb8622ef91930cde29b5/image
../data/OS-MN40-Miss/targ

../data/OS-MN40-Miss/target1/target/f654a77dd0f90111a3044518e0a403f8/image
../data/OS-MN40-Miss/target1/target/4b8d88c36f722762d4b8f83998caf6b3/image
../data/OS-MN40-Miss/target1/target/353265e2f527228392aaa2693d6a5e6a/image
../data/OS-MN40-Miss/target1/target/64333d0503dfc15735d52cb779da3eff/image
../data/OS-MN40-Miss/target1/target/ac8cfde0e276631f45106aa3f39ae0ff/image
../data/OS-MN40-Miss/target1/target/8785c0d5febde8ec69196e5a988fad98/image
../data/OS-MN40-Miss/target1/target/7265d5425b4826b44d73d25e11d54241/image
../data/OS-MN40-Miss/target1/target/e11b89b1cc95951a39c76267bc55bffe/image
../data/OS-MN40-Miss/target1/target/b6851b29f4cc049cf205e84520d39e28/image
../data/OS-MN40-Miss/target1/target/e6b8e68e8e919812e81d029b2f94825b/image
../data/OS-MN40-Miss/target1/target/cda8cb533cce17238ddd0a9ad5ed2652/image
../data/OS-MN40-Miss/target1/target/d343fba3a2b6a07e1b2c3b0bb17c296a/image
../data/OS-MN40-Miss/target1/target/4beed371309e81b2095760c9d848dc06/image
../data/OS-MN40-Miss/targ

../data/OS-MN40-Miss/target1/target/d84da7bbcc70bc460fb4ce31948011d9/image
../data/OS-MN40-Miss/target1/target/0e08b17de743bf2bfc16f4da5ef825b1/image
../data/OS-MN40-Miss/target1/target/5a0882fc2686fe45fa00b54fc0258fba/image
../data/OS-MN40-Miss/target1/target/8befde0de58b70d8eb0469318c57fc45/image
../data/OS-MN40-Miss/target1/target/2ac9aa5c71e5d39a763a317e9c8b5d45/image
../data/OS-MN40-Miss/target1/target/9df52d60c8083c5f3ceb9cd3474a7073/image
../data/OS-MN40-Miss/target1/target/7d5dfd667c014d919fa1cbdcb055ca97/image
../data/OS-MN40-Miss/target1/target/1d3bfd250b67c838ece13dff60eb01fe/image
../data/OS-MN40-Miss/target1/target/d43019050902ef4972131411a5feac12/image
../data/OS-MN40-Miss/target1/target/298a92f5fb6b07ba69e6d6fe1bf5ba6d/image
../data/OS-MN40-Miss/target1/target/5f53b31a40f90b9118f5792fd35356b1/image
../data/OS-MN40-Miss/target1/target/f7f9bf8987d5b71af253478624f7c3b9/image
../data/OS-MN40-Miss/target1/target/e42d4ffdd8d7c034afa31bbecb63396f/image
../data/OS-MN40-Miss/targ

../data/OS-MN40-Miss/target1/target/7e4c4c287247d25517c9ba0190997013/image
../data/OS-MN40-Miss/target1/target/af94eaf9707582359990a9db90c781bf/image
../data/OS-MN40-Miss/target1/target/aa0ab26d6c7b0a4e9bc736579c7f7364/image
../data/OS-MN40-Miss/target1/target/29a3496734766e88065ac5b3d0068ad5/image
../data/OS-MN40-Miss/target1/target/cdc00d9229d4b28f8af7bea4493c3c04/image
../data/OS-MN40-Miss/target1/target/cd37b11fee790ad27189e042a1689076/image
../data/OS-MN40-Miss/target1/target/a1bd5c573d9781edee1a8ece6eb04d99/image
../data/OS-MN40-Miss/target1/target/8ed545420f418ab11ab6e00f0b045449/image
../data/OS-MN40-Miss/target1/target/ab78adb493577ee2e8225f57b3f71a50/image
../data/OS-MN40-Miss/target1/target/05f6cd0ce3aedd3d6e790c843bd278ac/image
../data/OS-MN40-Miss/target1/target/862ce6eb082255cfc07eebf4a659c917/image
../data/OS-MN40-Miss/target1/target/e511921ca86cb7d59348d862ddfedaed/image
../data/OS-MN40-Miss/target1/target/723151dbcafca928a4cbe5dc395cbf49/image
../data/OS-MN40-Miss/targ

../data/OS-MN40-Miss/target1/target/88687012b329c55e1419a304926024bb/image
../data/OS-MN40-Miss/target1/target/86edf024f4a4d0f353a816d7c824219e/image
../data/OS-MN40-Miss/target1/target/dd72bb6afd57cca881b57bd27aefc0f5/image
../data/OS-MN40-Miss/target1/target/7b89f049da07b11b21744b27a987e1e9/image
../data/OS-MN40-Miss/target1/target/20a815b7912112adb42b968abd14c56c/image
../data/OS-MN40-Miss/target1/target/22adaf79bb25b12fcd424a192f77b516/image
../data/OS-MN40-Miss/target1/target/9b6bfbfd7dcd482106f6bb3ef995f15c/image
../data/OS-MN40-Miss/target1/target/9984aa2ab75bdaa494ff038fedcf32a4/image
../data/OS-MN40-Miss/target1/target/e08e7a84a1a154f3436ee8be2222a5dd/image
../data/OS-MN40-Miss/target1/target/0007d938a23a1e92865374fed452a101/image
../data/OS-MN40-Miss/target1/target/b4451f230ef8827d386d0ea98820b8f4/image
../data/OS-MN40-Miss/target1/target/e6608aba377480392c40504b76448d13/image
../data/OS-MN40-Miss/target1/target/902c7169d2a39ca49cb3f1f9ffefc90a/image
../data/OS-MN40-Miss/targ

In [66]:
#convert voxel to mesh using pymeshlab filters
cnt=0
for file in os.listdir(path):
     for file2 in os.listdir(path+file):
            if os.path.isdir(path+file+"/"+file2+"/voxel")==True and os.path.isdir(path+file+"/"+file2+"/mesh")==False and os.path.isdir(path+file+"/"+file2+"/pointcloud")==False:
                cnt=cnt+1  
    
                os.mkdir(path+file+"/"+file2+"/mesh")
                #coversion code here
                with open(path+file+"/"+file2+"/voxel/vox_32.binvox", 'rb') as f:
                    model = binvox_rw.read_as_3d_array(f)
                bv = model.data
                verts, faces, normals, values = measure.marching_cubes(bv, 0)

                ms = pymeshlab.MeshSet()
                qs = pymeshlab.Mesh(vertex_matrix = verts, face_matrix = faces)
                ms.add_mesh(qs)
                ms.save_current_mesh(path+file+"/"+file2+"/mesh/mesh.obj")
                print(path+file+"/"+file2+"/mesh/mesh.obj")
                
print(print(f'total_geneated_mesh  {cnt}')

../data/OS-MN40-Miss/target1/target/bd33c9aae749f1976fae0bf1073a483f/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/9e1c14af37bf8194ebea7ebc44a84e5f/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/f3ad106692ffb379880c70e50184d874/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/419247bd4cd14a44521acbf4efa20b5c/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/9036e1f23092d2246fdc8b4327655568/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/c4244c7ed1e1e4e8b911f64a9f82192c/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/09a47637b70247b177b7a96954261496/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/52f09bd97bae2aaa514c96bae2da78bb/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/68916779a228f5a4194167b030d39c58/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/115aef1d72ab305df04816d4ff98edbc/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/494003ed70f4db9bf6e43b55450dc738/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/d66ab87aa5a488c00d8780eb83f23f9f/mesh/mesh.obj
../d

../data/OS-MN40-Miss/target1/target/3b2603b807e851d50589289c4c449f3f/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/6c019956b48e5d8336cb7722b37484d1/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/978f63948e4168d5ba7e63d26992ee97/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/d99b78e5a785eddb635a8aedd2672317/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/1642b05c9d772001d62560d77fbceaa9/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/66a5150bff82ff90fe638fb21537b3e9/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/ee6f6fc31da8f75d56b4f6093ce1c536/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/7b51a7ad3f2a24753b6767010217cf07/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/f4f23e2dd83a5594eb488d99bd179191/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/8c77cc67f236494d461117ac825e1045/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/d892fc8975b994f3643f48d724ad09d7/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/b20bfc8432d5e0639837516edfc98b98/mesh/mesh.obj
../d

../data/OS-MN40-Miss/target1/target/b562c1c62364da45ad05e01ffab265ae/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/d02dfe8842bec0e93706c87c553db7fa/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/41b262406114dec12874dbb8bfe6e406/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/2c08cb795171d88092793ecd3cfa5fd4/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/7071202173ae3939e106aade14c9ad9c/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/b810e99f9fbd0b66992ee02a867855c6/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/0257dab7f87a9328b982425ddc9490e9/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/478ff8a96f7f9381e31b7c6463da4e35/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/28a6505633d470df1c0f6a72accdb07d/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/7a401a597a20f9069f62f799ed839977/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/1f4ae541d7983fcaafba00ced6ee0b4c/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/22dc2a836d44dae48f2048cb10b23dd4/mesh/mesh.obj
../d

../data/OS-MN40-Miss/target1/target/5f2916557b1e44182b14a62a2e154eb2/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/5fe4263d330c442e6aa601ad3ff7cce9/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/00be604ed20009fe36b1f5bb6e901bc3/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/a5c626940fff242a936c45041dc5199a/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/aafb7a84b8a6746c30b7a47c4ab6a71a/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/69a5a78385faa9d2f05ee069f397a3eb/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/aaa9bb90eda33c67691071f1e6bdde08/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/7dbb872095f67be22da52afcc876d290/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/0f0650f6ac02bfe5d938c9cf12c30622/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/3a688114075bc258883be959d2b82b02/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/1c3aab3ffd6c0bc30fb202d7dd41f845/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/e3608c6545f350eda4d0a09b76e4eaad/mesh/mesh.obj
../d

../data/OS-MN40-Miss/target1/target/0e36aacd06ec871d8435a799e4bc36d0/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/97a065787a487c7e206413083ea50f84/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/3463f0f5a25133c32f0dd49db0c3430b/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/b013c7238d1b27cb7f9f964cdbfabb89/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/ca9644f8010f434c414e66d16e5ce2a3/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/8f38c5f97e701f5b8dcf52ddf141428e/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/a4af8db919b73e11d25680e1b6caee92/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/4ddcc665a14acba58680058cd798c95d/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/36585d748890a13bf259d38ee5036415/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/653c29a49fba3197e425e02f3e1cbaf0/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/fb713cd92dd07174b2c2aaaa380b067e/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/42698f50f9897c70b9212e92eb0f0579/mesh/mesh.obj
../d

../data/OS-MN40-Miss/target1/target/5a509a60d4e594050ca5b5bca9a6fbb7/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/3e5ac8bb409b2ca0faef21c9fb3e2567/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/67ee5bcb5a5cd1ef98bba2c3ed27eba9/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/df82ba8a957434acaeabc27281f7d387/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/6fe9cfee8b771dfe79830838a66be87d/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/97844dd439bbc89f8f8a19164f576e18/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/176b6675a34c0082b82fdd65b588c369/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/16c911a5c90eaf408b61ba0648757b9e/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/070a45e5d43c16986848cce4aae03f8d/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/07d7e4ae8c76ce0250a770f51100806e/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/93f6caf5fbc59b2ad90dfcb4ba776ca3/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/e5a4604e0388026fb69bfe1ae641ae29/mesh/mesh.obj
../d

../data/OS-MN40-Miss/target1/target/a022359bf17a25d361d33d2231951c05/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/8e61d477c5f404403704687bae00e68d/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/e504d5139ed8ad55e3fc278289376d07/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/fbeac9075b526a2efcb2b83767d59afd/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/aabd8bae6c849f198adf929e2405e1f1/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/fd8be88541ec3ec1e120e1245723d219/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/1c67628fdb4043832f78f7d30171b215/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/e14483ca9339a6e947b2cf6d31b1dbc8/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/126d30c9719a8734c97afc92a9194fcb/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/10c954e8b1e6c97294892a6e1de2d3c1/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/2065b5b115eda06e30c9984568a608e1/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/fe66811d41bdae10c37377e87e52114d/mesh/mesh.obj
../d

../data/OS-MN40-Miss/target1/target/fc577e49365801d8efb9124a47cc6ade/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/39a0c6f260b2466249ab2513c11744ba/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/73fc33fbafa789fa53aee497674f9205/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/8dccab8b6d913db16e96f3aa7a631924/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/db6bfe4d1010b4de1e3437b6c7da5998/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/b91519cd50638b5ce9e5ba15e0d990ef/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/ada66506fee108cbbc1a975fbbb38600/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/8d335f8cf2d79cd7342c04496a2b5dd9/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/235f285607f5641ca12836734b67b241/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/451d9cae8c3d583f8556d57d4513c57a/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/6a2356f62b5299e75fa052eb3cf25c15/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/1d8ec116b33ec69697114ce2d38c3123/mesh/mesh.obj
../d

../data/OS-MN40-Miss/target1/target/a3e7fe34cf334a96f758c7068e405a7d/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/bb5561958b944bcffe37374904ec5404/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/3143585848d0a183d9f567f1f786147a/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/f43f2f0f3a06d4f712600b7c7ae81285/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/eab0c48330f9764c89c3f7bfb8881a4b/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/41b41f87bb70a06cf7cc05956157e2be/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/fe307b5e83111ca90aa8cfc8b0e1c2fc/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/6f9fff27a1476a886650625ee00d678b/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/f13883bd8e615673bb551f63acf79f5d/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/e55f4b96e2d50cf8701e7c635c9cf44b/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/699198e9db19ddca547d099f6f17af31/mesh/mesh.obj
../data/OS-MN40-Miss/target1/target/4e19bd288f29fb5b42888174228b8f81/mesh/mesh.obj
../d

In [5]:
#generation of multi-view images using mesh modality
cnt=0
for file in os.listdir(path):
     for file2 in os.listdir(path+file):
            if os.path.isdir(path+file+"/"+file2+"/mesh")==True and os.path.isdir(path+file+"/"+file2+"/image")==False:
                cnt=cnt+1  
                os.mkdir(path+file+"/"+file2+"/image")
                print(f'{path+file+"/"+file2+"/image"})
            
                #conversion code
                for i in range(24):
                    mesh2 = o3d.io.read_triangle_mesh(path+file+"/"+file2+"/mesh/mesh.obj")
                    mesh3 = o3d.io.read_triangle_mesh(path+file+"/"+file2+"/mesh/mesh.obj")
                    mesh2.compute_vertex_normals()
                    mesh3.compute_vertex_normals()
                    R1 = mesh2.get_rotation_matrix_from_xyz((0, 0, -0.5*(np.pi)))
                    mesh3.rotate(R1, center = (0, 0, 0))
                    #o3d.visualization.draw_geometries([mesh3])
                    R2 = mesh2.get_rotation_matrix_from_xyz(( -0.5*(np.pi), 0, 0))
                    mesh3.rotate(R2, center = (0, 0, 0))
                    #o3d.visualization.draw_geometries([mesh3])
                    R3 = mesh2.get_rotation_matrix_from_xyz((0, -2*i*(np.pi/24), 0))
                    mesh3.rotate(R3, center = (0, 0, 0))
                    #o3d.visualization.draw_geometries([mesh3])
                    R4 = mesh2.get_rotation_matrix_from_xyz((0.1666*(np.pi), 0, 0))
                    mesh3.rotate(R4, center = (0, 0, 0))
                    #o3d.visualization.draw_geometries([mesh3])
                    save_pic(mesh3,i,path+file+"/"+file2+"/image")
                
print(f'total_geneated_multi-view images {cnt}')

0


In [ ]:
############## GENERATION OF OTHER MODALITIES ##############
NOT REQUIRED FOR PRESENTED TASK since we are using pointcloud and image modalities 
can use below code to generate other two modalites

In [ ]:
# point-cloud to mesh generation 
# cnt=0
# for file in os.listdir(path):
#      for file2 in os.listdir(path+file):
#             if os.path.isdir(path+file+"/"+file2+"/pointcloud")==True and os.path.isdir(path+file+"/"+file2+"/mesh")==False:
#                 cnt=cnt+1
#                 os.mkdir(path+file+"/"+file2+"/mesh")#create directory 
#                 print(path+file+"/"+file2+"/mesh")
#                 #point cloud to mesh conversion
#                 ms = pymeshlab.MeshSet()
#                 ms.load_new_mesh(path+file+"/"+file2+"/pointcloud/pt_2048.txt",strformat='X Y Z' ,separator='SPACE')
#                 #ms.load_filter_script("point_to_image.mlx")#enter script here script (create solid fram)
#                 ms.surface_reconstruction_ball_pivoting()
#                 ms.save_current_mesh(path+file+"/"+file2+"/mesh/mesh.obj")
                   
#print(f'total_geneated_pointclouds  {cnt}')

In [ ]:
# #mesh to pointcloud geneartion :

# cnt=0
# for file in os.listdir(path):
#      for file2 in os.listdir(path+file):
#             if os.path.isdir(path+file+"/"+file2+"/mesh")==True and os.path.isdir(path+file+"/"+file2+"/pointcloud")==False:
#                 cnt=cnt+1
#                 os.mkdir(path+file+"/"+file2+"/pointcloud")
                
#                 #conversion code
#                 mesh = o3d.io.read_triangle_mesh(path+file+"/"+file2+"/mesh/mesh.obj")
#                 mesh.compute_vertex_normals()
#                 pcd1=mesh.sample_points_uniformly(number_of_points=1024)
#                 pcd2=mesh.sample_points_uniformly(number_of_points=2048)
#                 o3d.io.write_pointcloud(path+file+"/"+file2+"/pointcloud/pt_1024.xyz",pcd1)
#                 p=Path(path+file+"/"+file2+"/pointcloud/pt_1024.xyz")
#                 p.rename(p.with_suffix('.txt'))
#                 o3d.io.write_pointcloud(path+file+"/"+file2+"/pointcloud/pt_2048.xyz",pcd2)
#                 p=Path(path+file+"/"+file2+"/pointcloud/pt_2048.xyz")
#                 p.rename(p.with_suffix('.txt'))
#                 print(path+file+"/"+file2+"/pointclud/pt_2048.txt")
                               
# print(cnt)


In [ ]:
# #point-->mesh-->voxels the intermediate mesh is generated from poincloud to convert mesh into binvox of shape 32 32 32
# cnt=0
# for file in os.listdir(path):
#      for file2 in os.listdir(path+file):
#             if os.path.isdir(path+file+"/"+file2+"/pointcloud")==True and os.path.isdir(path+file+"/"+file2+"/voxel")==False:
#                 cnt=cnt+1  
#                 os.mkdir(path+file+"/"+file2+"/voxel")
                
#                 #conversion code here
#                 ms = pymeshlab.MeshSet()
#                 ms.load_new_mesh(path+file+"/"+file2+"/pointcloud/pt_2048.txt",strformat='X Y Z' ,separator='SPACE')
#                 #ms.load_filter_script("point_to_image.mlx")#enter script here script (create solid fram)
#                 ms.surface_reconstruction_ball_pivoting()
#                 ms.create_solid_wireframe()
#                 ms.set_current_mesh(new_curr_id=1)
#                 ms.save_current_mesh(path+file+"/"+file2+"/voxel/vox_32.obj") 
#                 #os cammand
#                 os.system(f"./binvox -d 32 -cb -pb {path+file+"/"+file2+"/voxel/vox_32.obj"}")
#                 os.remove(path+file+"/"+file2+"/voxel/vox_32.obj")
#                 print(path+file+"/"+file2+"/voxel/vox_32.obj")
                
# print(f'total_geneated_binvox_  {cnt}')